In [1]:
import copy
import random
from functools import partial

import numpy as np
from deap import algorithms, base, creator, tools, gp

In [2]:
class RobotController(object):
    def __init__(self, max_moves):
        self.max_moves = max_moves
        self.moves = 0
        self.consumed = 0
        self.routine = None
        
        self.direction = ['north', 'east', 'south', 'west']
        self.direction_row = [1, 0, -1, 0]
        self.direction_col = [0, 1, 0, -1]
        
    def _reset(self):
        self.row = self.row_start
        self.col = self.col_start
        self.direction = 1
        self.consumed = 0
        self.matric_exc = copy.deepcopy(self.matrix)
        
    def _conditional(self, condition, out1, out2):
        out1() if condition() else out2()
        
    def turn_left(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.direction = (self.direction - 1) % 4
            
    def turn_right(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.direction = (self.direction + 1) % 4
            
    def move_forward(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.row = (self.row + self.direction_row[self.direction]) % self.matrix_row
            self.col = (self.col + self.direction_col[self.direction]) % self.matrix_col
            
            if self.matrix_exc[self.row][self.col] == 'target':
                self.consumed += 1
                
            self.matrix_exc[self.row][self.col] = 'passed'
            
    def sense_target(self):
        ahead_row = (self.row + self.direction_row[self.direction]) % self.matrix_row
        ahead_col = (self.col + self.direction_col[self.direction]) % self.matrix_col
        
        return self.matrix_exc[ahead_row][ahead_col] == 'target'
    
    def if_target_ahead(self, out1, out2):
        return partial(self._conditional, self.sense_target, out1, out2)
    
    def run(self, routine):
        self._reset()
        while self.moves < self.max_moves:
            routine()
            
    def traverse_map(self, matrix):
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == '#':
                    self.matrix[-1].append('target')
                elif col == '.':
                    self.matrix[-1].append('empty')
                elif col == 'S':
                    self.matrix[-1].append('empty')
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.direction = 1
                    
        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        self.matrix_exc = copy.deepcopy(self.matrix)

In [3]:
class Prog(object):
    def _progn(self, *args):
        for arg in args:
            arg()
    
    def prog2(self, out1, out2):
        return partial(self._progn, out1, out2)
    
    def prog3(self, out1, out2, out3):
        return partial(self._progn, out1, out2, out3)

In [9]:
def eval_func(individual):
    global robot, pset
    
    #Transform the tree expression to functional Python code.
    routine = gp.compile(individual, pset)
    
    #Run the generated routine
    robot.run(routine)
    return robot.consumed,

def create_toolbox():
    global robot, pset
    
    pset = gp.PrimitiveSet('MAIN', 0)
    pset.addPrimitive(robot.if_target_ahead, 2)
    pset.addPrimitive(Prog().prog2, 2)
    pset.addPrimitive(Prog().prog3, 3)
    
    pset.addTerminal(robot.move_forward)
    pset.addTerminal(robot.turn_left)
    pset.addTerminal(robot.turn_right)
    
    creator.create('FitnessMax', base.Fitness, weights = (1.0,))
    creator.create('Individual', gp.PrimitiveTree, fitness = creator.FitnessMax)
    
    toolbox = base.Toolbox()
    
    toolbox.register('expr_init', gp.genFull, pset = pset, min_ = 1, max_ = 2)
    toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr_init)
    toolbox.register('population', tools.initRepeat, list, toolbox.individual)
    
    toolbox.register('evaluate', eval_func)
    toolbox.register('select', tools.selTournament, tournsize = 7)
    toolbox.register('mate', gp.cxOnePoint)
    toolbox.register('expr_mut', gp.genFull, min_ = 0, max_ = 2)
    toolbox.register('mutate', gp.mutUniform, expr = toolbox.expr_mut, pset = pset)
    
    return toolbox

In [10]:
global robot

random.seed(7)

In [18]:
max_moves = 750
robot = RobotController(max_moves)

toolbox = create_toolbox()

with open('data/target_map.txt') as f:
    robot.traverse_map(f)

population = toolbox.population(n = 400)
hall_of_fame = tools.HallOfFame(1)

C:\Users\U\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\U\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [19]:
#Register the stats
stats = tools.Statistics(lambda x: x.fitness.values)

stats.register('avg', np.mean)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('max', np.max)

In [20]:
#Define parameters
probab_crossover = 0.4
probab_mutate = 0.3
num_generations = 50

In [21]:
#Run the algorithm to solve the problem
algorithms.eaSimple(population, toolbox, probab_crossover, probab_mutate, num_generations, stats, halloffame = hall_of_fame)

gen	nevals	avg	std	min	max
0  	400   	0  	0  	0  	0  
1  	208   	0  	0  	0  	0  
2  	219   	0  	0  	0  	0  
3  	238   	0  	0  	0  	0  
4  	234   	0  	0  	0  	0  
5  	243   	0  	0  	0  	0  
6  	217   	0  	0  	0  	0  
7  	246   	0  	0  	0  	0  
8  	237   	0  	0  	0  	0  
9  	218   	0  	0  	0  	0  
10 	234   	0  	0  	0  	0  
11 	226   	0  	0  	0  	0  
12 	226   	0  	0  	0  	0  
13 	225   	0  	0  	0  	0  
14 	223   	0  	0  	0  	0  
15 	251   	0  	0  	0  	0  
16 	235   	0  	0  	0  	0  
17 	238   	0  	0  	0  	0  
18 	226   	0  	0  	0  	0  
19 	222   	0  	0  	0  	0  
20 	242   	0  	0  	0  	0  
21 	228   	0  	0  	0  	0  
22 	234   	0  	0  	0  	0  
23 	240   	0  	0  	0  	0  
24 	232   	0  	0  	0  	0  
25 	238   	0  	0  	0  	0  
26 	225   	0  	0  	0  	0  
27 	219   	0  	0  	0  	0  
28 	216   	0  	0  	0  	0  
29 	236   	0  	0  	0  	0  
30 	224   	0  	0  	0  	0  
31 	239   	0  	0  	0  	0  
32 	236   	0  	0  	0  	0  
33 	220   	0  	0  	0  	0  
34 	261   	0  	0  	0  	0  
35 	237   	0  	0  	0  	0  
3

([[<deap.gp.Primitive at 0x4d21310>,
 [{'gen': 0, 'nevals': 400, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 1, 'nevals': 208, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 2, 'nevals': 219, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 3, 'nevals': 238, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 4, 'nevals': 234, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 5, 'nevals': 243, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 6, 'nevals': 217, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 7, 'nevals': 246, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 8, 'nevals': 237, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 9, 'nevals': 218, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 10, 'nevals': 234, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 11, 'nevals': 226, 'avg': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0},
  {'gen': 12, 'nevals': 226, 'avg': 0